In [1]:
import tensorflow as tf

In [7]:
from PIL import Image, ImageDraw
import numpy as np
import os

In [3]:
# Load the model
model_path = r'C:\Users\Haroon Khan Awan\faster_rcnn_resnet101_latest\saved_model'  # Path to the directory containing the .index and .pb files
model = tf.saved_model.load(model_path)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [4]:
# Load and preprocess the image
image_path = r'C:\Users\Haroon Khan Awan\Dataset_Final\test\123123123.JPG'
image = Image.open(image_path)
desired_size = (224, 224)
image = image.resize(desired_size)
image_array = np.array(image, dtype=np.uint8)

# Convert the image array to a TensorFlow tensor
img_tensor = tf.convert_to_tensor(image_array, dtype=tf.uint8)

# Add an extra dimension to the tensor to represent batch size (if required)
img_tensor = tf.expand_dims(img_tensor, axis=0)

# Run the inference
infer = model.signatures["serving_default"]
detections = infer(img_tensor)

In [5]:
#Use this if you want to show the image as a popup
import cv2

# Get the bounding boxes, classes, and scores from the detections
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# Draw bounding boxes on the image
image_with_boxes = np.copy(image_array)
image_with_boxes = cv2.cvtColor(image_with_boxes, cv2.COLOR_RGB2BGR)
height, width, _ = image_with_boxes.shape

for i, box in enumerate(boxes):
    if scores[i] > 0.5:
        ymin, xmin, ymax, xmax = box
        x1 = int(xmin * width)
        y1 = int(ymin * height)
        x2 = int(xmax * width)
        y2 = int(ymax * height)
        cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image_with_boxes, str(classes[i]), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image with bounding boxes
cv2.imshow('Image with Bounding Boxes', image_with_boxes)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [8]:
#Use this if you want to save the original and predicted image

# Get the bounding boxes, classes, and scores from the detections
boxes = detections['detection_boxes'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(np.int32)
scores = detections['detection_scores'][0].numpy()

# Draw bounding boxes on the image
image_with_boxes = image.copy()
draw = ImageDraw.Draw(image_with_boxes)
width, height = image_with_boxes.size

for i, box in enumerate(boxes):
    if scores[i] > 0.5:
        ymin, xmin, ymax, xmax = box
        x1 = int(xmin * width)
        y1 = int(ymin * height)
        x2 = int(xmax * width)
        y2 = int(ymax * height)
        draw.rectangle([(x1, y1), (x2, y2)], outline='green')
        draw.text((x1, y1 - 10), str(classes[i]), fill='green')

# Save the original image
image.save(r'C:\Users\Haroon Khan Awan\Downloads\Dataset_Final\original_image.jpg')

# Save the image with bounding boxes
image_with_boxes.save(r'C:\Users\Haroon Khan Awan\Downloads\Dataset_Final\predicted_image.jpg')

In [12]:


# Path to the folder containing the images
image_folder = r'C:\Users\Haroon Khan Awan\Downloads\Dataset_Final\testest'

# Path to save the output images
output_folder = r'C:\Users\Haroon Khan Awan\Downloads\Dataset_Final\testest1'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Loop through the images in the folder
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Load and preprocess the image
        image_path = os.path.join(image_folder, filename)
        image = Image.open(image_path)
        desired_size = (224, 224)
        image = image.resize(desired_size)

        # Convert the image array to a TensorFlow tensor
        img_tensor = tf.convert_to_tensor(np.array(image), dtype=tf.uint8)

        # Add an extra dimension to the tensor to represent batch size (if required)
        img_tensor = tf.expand_dims(img_tensor, axis=0)

        # Run the inference
        infer = model.signatures["serving_default"]
        detections = infer(img_tensor)

        # Process the output detections
        # detections contains the predicted bounding boxes, class labels, and confidence scores

        # Get the bounding boxes, classes, and scores from the detections
        boxes = detections['detection_boxes'][0].numpy()
        classes = detections['detection_classes'][0].numpy().astype(np.int32)
        scores = detections['detection_scores'][0].numpy()

        # Draw bounding boxes on the image
        image_with_boxes = image.copy()
        draw = ImageDraw.Draw(image_with_boxes)
        width, height = image_with_boxes.size

        for i, box in enumerate(boxes):
            if scores[i] > 0.5:
                ymin, xmin, ymax, xmax = box
                x1 = int(xmin * width)
                y1 = int(ymin * height)
                x2 = int(xmax * width)
                y2 = int(ymax * height)
                draw.rectangle([(x1, y1), (x2, y2)], outline='green')
                draw.text((x1, y1 - 10), str(classes[i]), fill='green')

        # Save the original and predicted images side by side
        output_image_path = os.path.join(output_folder, filename)
        side_by_side_image = Image.new('RGB', (2 * desired_size[0], desired_size[1]))
        side_by_side_image.paste(image, (0, 0))
        side_by_side_image.paste(image_with_boxes, (desired_size[0], 0))
        side_by_side_image.save(output_image_path)
